In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

cols=['TRANSACTION_DATE','DOSAGE_UNIT','BUYER_COUNTY']
coldtypes={ 'DOSAGE_UNIT':np.float32 , 'BUYER_COUNTY':'category' }

change2year=lambda x : int(x[-4:])
#nationdfall=pd.read_csv('../prescription/arcos_all_washpost.tsv' , sep='\t', 
#                   usecols= cols,  dtype=coldtypes ,  engine='c',converters={ 'TRANSACTION_DATE':change2year } )

#nationdfall['TRANSACTION_DATE']=nationdfall['TRANSACTION_DATE'].astype('category')

#import pickle
#pickle.dump(nationdfall, open( "nation.pkl", "wb" ) )\

import pickle
with open('nation.pkl','rb') as pkldfile :
    ndf=pickle.load(pkldfile)

pivot4years={}

for cat in ndf.TRANSACTION_DATE.cat.categories:
    temp=ndf[ndf.TRANSACTION_DATE == cat ]
    pivot4years[str(cat)]=temp.pivot_table(values='DOSAGE_UNIT',aggfunc=np.sum,index=temp.BUYER_COUNTY,columns=temp.TRANSACTION_DATE ,fill_value=0)


import pickle
pickle.dump(pivot4years, open( "pivot4years.pkl", "wb" ) )

deaths_dtypes={'Year':int,'County':str,'Notes':str,'County Code':int,'Year Code':int , 'Deaths':pd.Int32Dtype(),'Population':int,'Crude Rate':float }
deaths=pd.read_csv('MCDNewEngland20062012.tsv',sep='\t',index_col=['County','Year'],dtype=deaths_dtypes, na_values=['Suppressed','Unreliable'])

with open('pivot4years.pkl','rb') as pkldfile :
    pivotByYear=pickle.load(pkldfile)

import geopandas as gpd
smallcounties = gpd.read_file('cb_2018_us_county_500k/cb_2018_us_county_500k.shp')

state_dict={ '25':'Massachusetts', '09':'Connecticut' , '23':'Maine', '33':'New Hampshire', '50':'Vermont' ,'44':'Rhode Island'  }
necounties=smallcounties[smallcounties['STATEFP'].isin(state_dict.keys())].copy()


In [6]:
geoOPyear=dict()
for i in pivotByYear.keys():
    geoOPyear[i]=necounties.merge(pivotByYear[i],left_on='NAME',right_on='BUYER_COUNTY')
    geoOPyear[i].drop(columns=['ALAND','AWATER'])

deathspivot=deaths['Deaths'].unstack()
deathspivot['CName']=deathspivot.index.str.split().str.get(0)
geoDeath=dict()
for i in geoOPyear.keys():
    temp=deathspivot[[int(i),'CName']]
    geoDeath[i]=geoOPyear[i].merge(temp,left_on='NAME',right_on='CName')
    geoDeath[i].drop(columns=['ALAND','AWATER'])
    


In [12]:
geoDeath.keys()

dict_keys(['2006', '2007', '2008', '2009', '2010', '2011', '2012'])

In [14]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012,CName
County,,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Addison
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN,Androscoggin
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aroostook
"Barnstable County, MA",22,29,22,20,20,16,19,Barnstable
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN,Belknap


In [15]:
deathspivot.columns

Index([2006, 2007, 2008, 2009, 2010, 2011, 2012, 'CName'], dtype='object', name='Year')

In [20]:
pivotByYear.keys()

dict_keys(['2006', '2007', '2008', '2009', '2010', '2011', '2012'])

In [24]:
pivotByYear['2007'].head()

TRANSACTION_DATE,2007
BUYER_COUNTY,
ADDISON,844900
AGUADA,49500
AGUADILLA,184800
AGUAS BUENAS,29400
ALACHUA,8139364


In [25]:
necounties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
56,09,009,00212798,0500000US09009,09009,New Haven,06,1565688367,667081242,"MULTIPOLYGON (((-72.76143 41.24233, -72.75973 ..."
142,23,013,00581292,0500000US23013,23013,Knox,06,945684684,2017116243,"MULTIPOLYGON (((-68.55926 44.04502, -68.55504 ..."
143,23,015,00581293,0500000US23015,23015,Lincoln,06,1180688991,631299424,"MULTIPOLYGON (((-69.32357 43.75899, -69.32114 ..."
270,25,001,00606927,0500000US25001,25001,Barnstable,06,1021048430,2360287543,"POLYGON ((-70.68698 41.52949, -70.68327 41.532..."
271,25,013,00606933,0500000US25013,25013,Hampden,06,1597996708,44417070,"POLYGON ((-73.07484 42.10615, -73.07305 42.106..."
